<a href="https://colab.research.google.com/github/NickLarsonUVA/DS2002/blob/main/Finance_API_Tutorial_Homework_Assignment_%E2%80%93_Work_Alone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [59]:
import json
import pandas as pd
import json
import requests

Task 1: Fetch Basic Stock Data 10pts
- Prompt the user to input two or more stock symbols.
- Use the API to fetch data for the given symbols.
- Extract and display only the Stock Ticker, Company Name, and Current Market Price

In [60]:
#Enter two or more stocks into the input (using tickers as the input)
stock = input()

AAPL


In [65]:
apikey="" #use your own key

url = "https://yfapi.net/v6/finance/quote"

querystring = {"symbols":stock}

headers = {
    'x-api-key': apikey
    }

response = requests.request("GET", url, headers=headers, params=querystring)

print(response.text)

{"quoteResponse":{"result":[{"language":"en-US","region":"US","quoteType":"EQUITY","typeDisp":"Equity","quoteSourceName":"Nasdaq Real Time Price","triggerable":true,"customPriceAlertConfidence":"HIGH","currency":"USD","regularMarketChangePercent":-2.7040126,"regularMarketPrice":240.36,"exchangeTimezoneName":"America/New_York","exchangeTimezoneShortName":"EST","gmtOffSetMilliseconds":-18000000,"market":"us_market","esgPopulated":false,"corporateActions":[],"longName":"Apple Inc.","postMarketTime":1740610387,"regularMarketTime":1740603601,"exchange":"NMS","messageBoardId":"finmb_24937","shortName":"Apple Inc.","forwardPE":28.924187,"priceToBook":54.15953,"sourceInterval":15,"exchangeDataDelayedBy":0,"averageAnalystRating":"2.1 - Buy","tradeable":false,"cryptoTradeable":false,"hasPrePostMarketData":true,"firstTradeDateMilliseconds":345479400000,"priceHint":2,"postMarketChangePercent":-0.09152988,"postMarketPrice":240.14,"postMarketChange":-0.22000122,"regularMarketChange":-6.6799927,"regu

In [66]:
df = pd.DataFrame(response.json()['quoteResponse']['result'])

# Extract Market Price and format it
df['Market Price'] = df['regularMarketPrice'].apply(lambda x: f"${x:,.2f}" if pd.notna(x) else "N/A")

# Display the results
print(df[['symbol', 'shortName', 'Market Price']])

  symbol   shortName Market Price
0   AAPL  Apple Inc.      $240.36


Task 2.1: Fetch Additional Data Using Modules 10pts
- Allow the user to choose a module from the Quote Summary Endpoint.
Find:
52 Week High
52 Week Low
Return on Assets (ROA)
- Make an API request using the chosen module.
- Convert the response into a Pandas DataFrame.
- Display the DataFrame.

In [67]:
import requests
import pandas as pd

def get_finance_data():
    apikey = ""  # Use your own key
    stock = input("Enter the stock symbol: ")

    modules = {
        "1": "summaryDetail",
        "2": "financialData"
    }

    print("Choose a module:")
    print("1: 52 Week High / 52 Week Low")
    print("2: Return on Assets (ROA)")

    choice = input("Enter the number of your choice: ")
    module = modules.get(choice)

    if not module:
        print("Invalid choice.")
        return

    url = f"https://yfapi.net/v11/finance/quoteSummary/{stock}"
    querystring = {"modules": module}
    headers = {
        'x-api-key': apikey
        }

    response = requests.get(url, headers=headers, params=querystring)

    # Extract relevant data based on module choice
    result = response.json()['quoteSummary']['result'][0]

    if choice == "1":
        data = {
            "52 Week High": result["summaryDetail"]["twoHundredDayAverage"]["raw"],
            "52 Week Low": result["summaryDetail"]["fiftyTwoWeekLow"]["raw"]
        }
    elif choice == "2":
        data = {
            "ROA": result["financialData"]["returnOnAssets"]["raw"]
        }

    # Convert the extracted data into a pandas DataFrame
    df = pd.DataFrame([data])

    print(df)

get_finance_data()

Enter the stock symbol: AAPL
Choose a module:
1: 52 Week High / 52 Week Low
2: Return on Assets (ROA)
Enter the number of your choice: 1
   52 Week High  52 Week Low
0      225.0356       164.08


Task 2.2: Fetch Additional Data Using Modules 10pts
Find Current Trending Stocks , Their Actual Name and Ticker and Current Price as well as
their 52 high and low.

In [70]:
import pandas as pd
import requests

def get_trending_stocks():
    apikey = ""  # Use your own key

    trending_url = "https://yfapi.net/v1/finance/trending/US"
    headers = {'x-api-key': apikey}

    trending_response = requests.get(trending_url, headers=headers)
    trending_data = trending_response.json().get('finance', {}).get('result', [{}])[0].get('quotes', [])

    if not trending_data:
        print("No trending stocks found.")
        return

    symbols = ",".join([stock['symbol'] for stock in trending_data])

    #using the code from part 1 of the assignment
    quote_url = "https://yfapi.net/v6/finance/quote"
    querystring = {"symbols": symbols}

    quote_response = requests.get(quote_url, headers=headers, params=querystring)
    quote_data = quote_response.json().get('quoteResponse', {}).get('result', [])

    df = pd.DataFrame(quote_data)
    df['Market Price'] = df['regularMarketPrice'].apply(lambda x: f"${x:,.2f}" if pd.notna(x) else "N/A")

    # adding the additional data for 52-week high/low based on 2.1
    for stock in quote_data:
        stock_symbol = stock['symbol']
        summary_url = f"https://yfapi.net/v11/finance/quoteSummary/{stock_symbol}"
        summary_querystring = {"modules": "summaryDetail"}
        summary_response = requests.get(summary_url, headers=headers, params=summary_querystring)
        summary_data = summary_response.json().get('quoteSummary', {}).get('result', [{}])[0].get('summaryDetail', {})

        df.loc[df['symbol'] == stock_symbol, '52 Week High'] = summary_data.get('twoHundredDayAverage', {}).get('raw', "N/A")
        df.loc[df['symbol'] == stock_symbol, '52 Week Low'] = summary_data.get('fiftyTwoWeekLow', {}).get('raw', "N/A")

    print(df[['symbol', 'shortName', 'Market Price', '52 Week High', '52 Week Low']])

# Run the function to display trending stocks
get_trending_stocks()

   symbol                        shortName Market Price  52 Week High  \
0    NVDA               NVIDIA Corporation      $131.28     126.19863   
1     CRM                 Salesforce, Inc.      $307.33     289.40564   
2    SNOW                   Snowflake Inc.      $166.19     141.79974   
3    IONQ                       IonQ, Inc.       $29.93      18.85830   
4    BYND                Beyond Meat, Inc.        $3.56       5.72290   
5     APP             Applovin Corporation      $331.00     192.46867   
6    EBAY                        eBay Inc.       $69.14      60.29120   
7      AI                      C3.ai, Inc.       $26.44      28.84715   
8     NIO                         NIO Inc.        $4.72       4.75385   
9    SNPS                   Synopsys, Inc.      $470.46     535.60830   
10   MULN          Mullen Automotive, Inc.        $4.04    5792.82030   
11   NTNX                    Nutanix, Inc.       $69.35      61.52975   
12   MARA              MARA Holdings, Inc.       $1

Bonus Challenge
- Allow users to fetch data for different modules dynamically. (revised some of the code to allow users to input there own)
- Format currency values properly (e.g., `$1,500,000,000` instead of `1500000000`(did this with the lambda functions